In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from timeit import timeit
import gudhi as gd
from torchvision.datasets import MNIST, KMNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from eclayr.cubical import CubicalEcc, ECLayr

batch_size = 32
num_iter = 10

# MNIST

In [3]:
train_data = MNIST(root="./MNIST/dataset/raw/", train=True, download=True, transform=ToTensor())  # shape: (60000, 28, 28)
X = (train_data.data / 255).unsqueeze(1)
dataloader = DataLoader(X, batch_size=batch_size, shuffle=True)

## ECC

### T-construction

In [12]:
cubecc = CubicalEcc(t_const=True, sublevel=False, interval=[0, 1])

def train():
    for batch in dataloader:
        ecc = cubecc(batch)

runtime = timeit(train, number=num_iter)
print("Runtime for each iteration:", runtime / num_iter)

Runtime for each iteration: 12.716819541000177


### V-construction

In [27]:
cubecc = CubicalEcc(t_const=False, sublevel=False, interval=[0, 1])

def train():
    for batch in dataloader:
        ecc = cubecc(batch)

runtime = timeit(train, number=num_iter)
print("Runtime for each iteration:", runtime / num_iter)

Runtime for each iteration: 2.8620287499943515


## ECC with gradient

In [28]:
X.requires_grad_(True)
dataloader = DataLoader(X, batch_size=batch_size, shuffle=True)

### T-construction

In [30]:
cubecc = CubicalEcc(t_const=True, sublevel=False, interval=[0, 1])

def train():
    for batch in dataloader:
        ecc = cubecc(batch)

runtime = timeit(train, number=num_iter)
print("Runtime for each iteration:", runtime / num_iter)

Runtime for each iteration: 16.93060333399626


### V-construction

In [31]:
cubecc = CubicalEcc(t_const=False, sublevel=False, interval=[0, 1])

def train():
    for batch in dataloader:
        ecc = cubecc(batch)

runtime = timeit(train, number=num_iter)
print("Runtime for each iteration:", runtime / num_iter)

Runtime for each iteration: 9.896869666001294


## PH

In [4]:
X.requires_grad_(False)
dataloader = DataLoader(X, batch_size=batch_size, shuffle=True)

### T-construction

In [8]:
def train():
    for batch in dataloader:
        for data in batch:
            data = -data    # bc. superlevel set is used
            for channel in data:
                cpx = gd.CubicalComplex(top_dimensional_cells=channel)
                # ph = cpx.persistence()

runtime = timeit(train, number=num_iter)
print("Runtime for each iteration:", runtime / num_iter)

Runtime for each iteration: 12.497989959000051


### V-construction

In [7]:
def train():
    for batch in dataloader:
        for data in batch:
            data = -data    # bc. superlevel set is used
            for channel in data:
                cpx = gd.CubicalComplex(vertices=channel)
                ph = cpx.persistence()

runtime = timeit(train, number=num_iter)
print("Runtime for each iteration:", runtime / num_iter)

Runtime for each iteration: 31.643859957999894


# KMNIST

In [4]:
train_data = KMNIST(root="./KMNIST/dataset/raw/", train=True, download=True, transform=ToTensor())  # shape: (60000, 28, 28)
X = (train_data.data / 255).unsqueeze(1)
dataloader = DataLoader(X, batch_size=batch_size, shuffle=True)

## ECC

### T-construction

In [ ]:
cubecc = CubicalEcc(t_const=True, sublevel=False, interval=[0, 1])

def train():
    for batch in dataloader:
        ecc = cubecc(batch)

runtime = timeit(train, number=num_iter)
print("Runtime for each iteration:", runtime / num_iter)

Runtime for each iteration: 12.716819541000177


### V-construction

In [5]:
cubecc = CubicalEcc(t_const=False, sublevel=False, interval=[0, 1])

def train():
    for batch in dataloader:
        ecc = cubecc(batch)

runtime = timeit(train, number=num_iter)
print("Runtime for each iteration:", runtime / num_iter)

Runtime for each iteration: 2.8850671000000148


## ECC with gradient

In [ ]:
X.requires_grad_(True)
dataloader = DataLoader(X, batch_size=batch_size, shuffle=True)

### T-construction

In [ ]:
cubecc = CubicalEcc(t_const=True, sublevel=False, interval=[0, 1])

def train():
    for batch in dataloader:
        ecc = cubecc(batch)

runtime = timeit(train, number=num_iter)
print("Runtime for each iteration:", runtime / num_iter)

Runtime for each iteration: 16.93060333399626


### V-construction

In [ ]:
cubecc = CubicalEcc(t_const=False, sublevel=False, interval=[0, 1])

def train():
    for batch in dataloader:
        ecc = cubecc(batch)

runtime = timeit(train, number=num_iter)
print("Runtime for each iteration:", runtime / num_iter)

Runtime for each iteration: 9.896869666001294


## PH

In [ ]:
X.requires_grad_(False)
dataloader = DataLoader(X, batch_size=batch_size, shuffle=True)

### T-construction

In [ ]:
def train():
    for batch in dataloader:
        for data in batch:
            data = -data    # bc. superlevel set is used
            for channel in data:
                cpx = gd.CubicalComplex(top_dimensional_cells=channel)
                # ph = cpx.persistence()

runtime = timeit(train, number=num_iter)
print("Runtime for each iteration:", runtime / num_iter)

Runtime for each iteration: 12.497989959000051


### V-construction

In [ ]:
def train():
    for batch in dataloader:
        for data in batch:
            data = -data    # bc. superlevel set is used
            for channel in data:
                cpx = gd.CubicalComplex(vertices=channel)
                ph = cpx.persistence()

runtime = timeit(train, number=num_iter)
print("Runtime for each iteration:", runtime / num_iter)

Runtime for each iteration: 31.643859957999894


# Grad analysis

In [8]:
data1 = X[:256].clone()
data1.requires_grad_(True)
data1.retain_grad()

cub1 = CubECC2d(as_vertices=False, sublevel=True, interval=[0, 1])
y1 = cub1(data1)
loss1 = y1.sum()
loss1.backward()

In [7]:
data2 = X[:256].clone()
data2.requires_grad_(True)
data2.retain_grad()

cub2 = CubECC2d_b(t_const=True, sublevel=True, interval=[0, 1])
y2 = cub2(data2)
loss2 = y2.sum()
loss2.backward()

In [9]:
(y1 == y2).all()

tensor(True)

In [10]:
import torch
torch.allclose(data1.grad, data2.grad, rtol=0, atol=0.00001)

True

In [11]:
(data1.grad - data2.grad).abs().max()

tensor(8.3447e-07)